# LOCI Demo UI

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

import pandas as pd
import pyLDAvis

import loci as lc
from loci import io
from loci import analytics
from loci import index
from loci import clustering
from loci import topics
from loci import plots

widgets.interact_manual.opts['manual_name'] = 'Execute'

In [ ]:
w_loc_filter = ''
w_osmwrangle = ''
w_clustering = ''
w_cluster_shapes = ''

## Load POIs from a remote file produced by OSMWrangle

### Define the area boundary

In [ ]:
def get_bound(location, buffer_dist):
    bound = lc.io.retrieve_osm_loc(location, buffer_dist)
    display(lc.plots.map_geometry(bound))
    return bound

w_loc_filter = interactive(get_bound,
                {'manual':True, 'manual_name':'Execute'},
                location='Athens, Greece',
                buffer_dist=(0,50000,100))
display(w_loc_filter)

### Load and filter the file

In [ ]:
def import_osmwrangle(file, target_crs):
    bound = w_loc_filter.result
    pois = lc.io.import_osmwrangle(file, bound=bound, target_crs=target_crs)
    display(pois.head())
    return pois

w_osmwrangle = interactive(import_osmwrangle,
                {'manual':True, 'manual_name':'Execute'},
                file='http://download.slipo.eu/results/osm-to-csv/europe/europe_greece-pois.osm.csv.zip',
                target_crs='EPSG:2100')
display(w_osmwrangle)

## Show a sample on the map

In [ ]:
@interact_manual(sample_size=(10,1000,10), show_bbox=True)
def sample_preview(sample_size, show_bbox):
    pois = w_osmwrangle.result
    display(lc.plots.map_points(pois.sample(sample_size), show_bbox=show_bbox))

## Plot the frequency of the POI keywords

In [ ]:
@interact_manual()
def kwds_freq():
    pois = w_osmwrangle.result
    kf = lc.analytics.kwds_freq(pois)
    lc.plots.barchart(kf, plot_title='Top Keywords', x_axis_label='Keywords', y_axis_label='Frequency')

## Generate a word cloud from the POI keywords

In [ ]:
@interact_manual()
def wordcloud():
    pois = w_osmwrangle.result
    lc.plots.plot_wordcloud(pois)

## Select a specific type of POIs and generate a heat map

In [ ]:
@interact_manual(kwd='shop', radius=(5,15,1))
def heatmap(kwd, radius):
    pois = w_osmwrangle.result
    if kwd is not None and kwd != '':
        pois_filtered = lc.analytics.filter_by_kwd(pois, kwd)
    else:
        pois_filtered = pois
    display(lc.plots.heatmap(pois_filtered, radius=radius))

## Plot a grid

In [ ]:
@interact_manual(ratio=(0.02,0.1,0.01))
def grid(ratio):
    pois = w_osmwrangle.result
    g, num_columns, num_rows = lc.index.grid(pois, cell_size_ratio=ratio)
    display(lc.plots.map_choropleth(areas=g, id_field='cell_id', value_field='score'))

## Compute clusters

In [ ]:
def clustering(alg, minPts, eps):
    pois = w_osmwrangle.result
    minPts = int(minPts)
    eps = float(eps)
    pois_in_clusters, eps_per_cluster = lc.clustering.compute_clusters(pois,
                                                                       alg=alg,
                                                                       min_pts=minPts,
                                                                       eps=eps,
                                                                       n_jobs=-1)
    return [pois_in_clusters, eps_per_cluster]

w_clustering = interactive(clustering,
                {'manual':True, 'manual_name':'Execute'},
                alg=['dbscan', 'hdbscan'],
                minPts='15',
                eps='100')
display(w_clustering)

## Show clusters on the map

In [ ]:
def cluster_shapes(type):
    pois_in_clusters = w_clustering.result[0]
    eps_per_cluster = w_clustering.result[1]
    cluster_borders = lc.clustering.cluster_shapes(pois_in_clusters, type, eps_per_cluster)
    display(plots.map_choropleth(cluster_borders, id_field='cluster_id', value_field='size'))
    return cluster_borders

w_cluster_shapes = interactive(cluster_shapes,
                {'manual':True, 'manual_name':'Execute'},
                type=[1,2,3])
display(w_cluster_shapes)

## Show the POIs of a specific cluster

In [ ]:
@interact_manual(cluster_id='0')
def show_pois_in_cluster(cluster_id):
    pois_in_clusters = w_clustering.result[0]
    selected_cluster = int(cluster_id)
    selected_pois = pois_in_clusters.loc[pois_in_clusters['cluster_id'] == selected_cluster]
    display(plots.map_points(selected_pois))

## Show OSM buildings and streets in a specific cluster

In [ ]:
@interact_manual(cluster_id='0')
def show_osm_features_in_cluster(cluster_id):
    pois_in_clusters = w_clustering.result[0]
    cluster_borders = w_cluster_shapes.result
    selected_cluster = int(cluster_id)
    selected_cluster = cluster_borders.loc[cluster_borders['cluster_id'] == selected_cluster]
    display(plots.map_cluster_contents_osm(selected_cluster))

## Compute AOIs for different POI types and compare them on the map

In [ ]:
@interact_manual(type_a='restaurant', type_b='tourism', alg=['dbscan', 'hdbscan'], min_pts='10', eps='100')
def cluster_compare(type_a, type_b, alg, min_pts, eps):
    pois = w_osmwrangle.result
    min_pts=int(min_pts)
    eps=float(eps)
    
    pois_a = lc.analytics.filter_by_kwd(pois, kwd_filter=type_a)
    pois_b = lc.analytics.filter_by_kwd(pois, kwd_filter=type_b)

    aois_a, eps_a = lc.clustering.compute_clusters(pois_a, alg=alg, min_pts=min_pts, eps=eps, n_jobs=-1)
    aois_b, eps_b = lc.clustering.compute_clusters(pois_b, alg=alg, min_pts=min_pts, eps=eps, n_jobs=-1)

    aois_a = lc.clustering.cluster_shapes(aois_a, 2, eps_a)
    aois_b = lc.clustering.cluster_shapes(aois_b, 2, eps_b)
    
    display(lc.plots.map_cluster_diff(aois_a, aois_b))

## Extract topics from the clusters' keywords

In [ ]:
def extract_topics(num_topics):
    pois_in_clusters = w_clustering.result[0]
    cluster_topics, topic_keywords, visualized_topics = lc.topics.topic_modeling(pois_in_clusters,
                                                                                 num_of_topics=num_topics,
                                                                                 kwds_per_topic=10)
    display(topic_keywords.head(num_topics))
    cluster_borders = w_cluster_shapes.result
    cluster_borders_topics = pd.merge(cluster_borders,
                                      cluster_topics,
                                      left_on='cluster_id',
                                      right_index=True,
                                      how='inner')
    return [cluster_topics, topic_keywords, visualized_topics, cluster_borders_topics]


w_topics = interactive(extract_topics,
                {'manual':True, 'manual_name':'Execute'},
                num_topics=(2,10,1))
display(w_topics)

## Display clusters with topics

In [ ]:
@interact_manual(viz_type=['dominant', 'single', 'rgb'], topic_id='')
def vis_topics(viz_type, topic_id):
    cluster_borders_topics = w_topics.result[3]
    if viz_type == 'single':
        topic_id = 'Topic' + topic_id
        display(lc.plots.map_clusters_with_topics(cluster_borders_topics, viz_type='single', single_topic=topic_id))
    elif viz_type == 'rgb':
        display(lc.plots.map_clusters_with_topics(cluster_borders_topics, viz_type='rgb', red='Topic0',
                                                  green='Topic1', blue='Topic2'))
    else:
        display(lc.plots.map_clusters_with_topics(cluster_borders_topics, viz_type='dominant'))